In [2]:
%pip install 'mosaicml[nlp, tensorboard]'
# To install from source instead of the last release, comment the command above and uncomment the following one.
# %pip install 'mosaicml[nlp, tensorboard] @ git+https://github.com/mosaicml/composer.git'"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [3]:
import transformers

# Create a BERT sequence classification model using Hugging Face transformers
model = transformers.AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader

db=load_dataset('dair-ai/emotion')
def tokenize_function(data):

    return tokenizer(
        data['text'],
        padding='max_length',
        max_length=256,
        truncation=True,
        return_tensors='pt'
    )

db_train=db['train']
db_eval=db['validation']
token_train=db_train.map(tokenize_function,batched=True,batch_size=100,remove_columns=['text'])
token_eval=db_eval.map(tokenize_function,batched=True,batch_size=100,remove_columns=['text'])

data_collator = transformers.data.data_collator.default_data_collator
train_loader = DataLoader(token_train, batch_size=16, shuffle=False, drop_last=False, collate_fn=data_collator)
eval_loader = DataLoader(token_eval,batch_size=16, shuffle=False, drop_last=False, collate_fn=data_collator)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
db_train_df=db_train.to_pandas()
db_train_df['label'].value_counts()
idx=db_train_df['label'].value_counts().index
label_pair={3:"anger",2:"love",5:"suprise",4:"fear",1:"joy",2:"love",0:"sadness"}
label2idx=[y for x,y in label_pair.items()]
label2idx

['anger', 'love', 'suprise', 'fear', 'joy', 'sadness']

In [6]:
from torchmetrics.classification import MulticlassAccuracy
from composer.models.huggingface import HuggingFaceModel
from composer.metrics import CrossEntropy

metrics = [CrossEntropy(), MulticlassAccuracy(num_classes=6, average='micro')]
# Package as a trainer-friendly Composer model
composer_model = HuggingFaceModel(model, tokenizer=tokenizer, metrics=metrics, use_logits=True)

In [7]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR

optimizer = AdamW(
    params=composer_model.parameters(),
    lr=3e-5, betas=(0.9, 0.98),
    eps=1e-6, weight_decay=3e-6
)
linear_lr_decay = LinearLR(
    optimizer, start_factor=1.0,
    end_factor=0, total_iters=150
)

In [8]:
token_eval

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})

In [17]:
eval_loader = DataLoader(token_eval,batch_size=16, shuffle=False, drop_last=False, collate_fn=data_collator)

In [9]:
import torch
from composer import Trainer

# Create Trainer Object
trainer = Trainer(
    model=composer_model, # This is the model from the HuggingFaceModel wrapper class.
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    max_duration="1ep",
    optimizers=optimizer,
    schedulers=[linear_lr_decay],
    device='gpu' if torch.cuda.is_available() else 'cpu',
    train_subset_num_batches=150,
    precision='fp32',
    seed=17
)
# Start training
trainer.fit()

******************************
Config:
composer_commit_hash: None
composer_version: 0.17.2
node_name: unknown because NODENAME environment variable not set
num_gpus_per_node: 1
num_nodes: 1
rank_zero_seed: 17

******************************


train          Epoch   0:    0%|| 0/150 [00:00<?, ?ba/s]         

eval           Epoch   0:    0%|| 0/125 [00:00<?, ?ba/s]         

In [10]:
trainer.state.eval_metrics

{'eval': {'CrossEntropy': CrossEntropy(),
  'MulticlassAccuracy': MulticlassAccuracy()}}

In [20]:
eval_batch = next(iter(eval_loader))

# Move batch to gpu
eval_batch = {k: v.cuda() if torch.cuda.is_available() else v for k, v in eval_batch.items()}
with torch.no_grad():
    predictions = composer_model(eval_batch)["logits"].argmax(dim=1)

# Visualize only 5 samples
predictions = predictions[:5]

label = label2idx #['anger', 'love', 'suprise', 'fear', 'joy', 'sadness']
for i, prediction in enumerate(predictions):
    sentence = db["validation"][i]["text"]
    correct_label = label[db["validation"][i]["label"]]
    prediction_label = label[prediction]
    print(f"Sample: {sentence}")
    print(f"Label: {correct_label}")
    print(f"Prediction: {prediction_label}")
    print()

Sample: im feeling quite sad and sorry for myself but ill snap out of it soon
Label: anger
Prediction: anger

Sample: i feel like i am still looking at a blank canvas blank pieces of paper
Label: anger
Prediction: anger

Sample: i feel like a faithful servant
Label: suprise
Prediction: love

Sample: i am just feeling cranky and blue
Label: fear
Prediction: fear

Sample: i can have for a treat or if i am feeling festive
Label: love
Prediction: love

